In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import torch.nn as nn
from transformers import AutoModel
from datasets import load_dataset
from transformers import AutoTokenizer
import random
import math
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data.distributed import DistributedSampler
from transformers import AutoModelForSeq2SeqLM
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import get_linear_schedule_with_warmup
torch.manual_seed(66)

Mounted at /content/drive


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
train_dataset = load_dataset("squad", split="train")

test_dataset = load_dataset("squad", split="validation")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
print(train_dataset)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})


In [6]:
print(train_dataset['id'][0])
print(train_dataset['title'][0])
print(train_dataset['context'][0])
print(train_dataset['question'][0])
print(train_dataset['answers'][0])

5733be284776f41900661182
University_of_Notre_Dame
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


In [7]:
print(test_dataset['id'][0])
print(test_dataset['title'][0])
print(test_dataset['context'][0])
print(test_dataset['question'][0])
print(test_dataset['answers'][0])

56be4db0acb8001400a502ec
Super_Bowl_50
Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Which NFL team represented the AFC at Super Bowl 50?
{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answer_start': [177, 177, 177]}


In [8]:
print(test_dataset['id'][2])
print(test_dataset['title'][2])
print(test_dataset['context'][2])
print(test_dataset['question'][2])
print(test_dataset['answers'][2])

56be4db0acb8001400a502ee
Super_Bowl_50
Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Where did Super Bowl 50 take place?
{'text': ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."], 'answer_start': [

In [9]:
model_name = 'facebook/bart-base'

#Data Tokenize and Preprocessing

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length = 512)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
# single_item = train_dataset[0]
# print(single_item)

# context = single_item['context'].strip()
# question = single_item['question'].strip()
# answer_text = single_item['answers']['text'][0].strip()
# question_tokens = tokenizer.tokenize(question)
# input_ids = tokenizer(question)['input_ids']

# answer_start = single_item['answers']['answer_start'][0]
# answer_end = answer_start + len(answer_text) - 1

# print(single_item['question'])
# print(question)
# print(question_tokens)
# print(input_ids)
# print(answer_text)
# print(context[answer_end])

In [11]:
def tokenize_single_sample(dict_row):
  context = dict_row['context'].strip()
  context_tokens = tokenizer.tokenize(context, truncation=True, max_length=512)

  question = dict_row['question'].strip()
  question_tokens = tokenizer.tokenize(question, truncation=True, max_length=512)
  question_input_ids = tokenizer(question, truncation=True, max_length=512)['input_ids']
  quesiton_input_positions = torch.tensor(list(range(len(question_input_ids))))

  answer_text = dict_row['answers']['text'][0].strip()
  answer_tokens = tokenizer.tokenize(answer_text, truncation=True, max_length=512)
  answer_start = dict_row['answers']['answer_start'][0]
  answer_end = answer_start + len(answer_text) - 1
  # answer token start and end position at the context token array
  answer_token_start = len(tokenizer(context[:answer_start], truncation=True, max_length=512)['input_ids']) - 1
  answer_token_end = len(tokenizer(context[:answer_end], truncation=True, max_length=512)['input_ids']) - 1


  # Section-1: Question Generation (Input: will be Context and Answer text, Output: Question)
  tokenized_input_qg = tokenizer(context +'</s><s>'+ answer_text, truncation=True, max_length=512)['input_ids']
  # task=0 for Question Generation, 1 for Question Answering and 2 for Uncommon Word generation
  task = 0
  tokenized_input_position_qg = torch.tensor(list(range(len(context_tokens)+2)) + list(range(len(answer_tokens)+2)))
  task_embedding_input_qg = torch.tensor([task] * len(tokenized_input_qg))
  # Segment ID = 0 for context, 1 for answer and 2 for question. and +2 is for token <s> and </s> for each segment
  segment_embedding_input_qg = torch.tensor([0] * (len(context_tokens)+2) + [1] * (len(answer_tokens)+2))

  # Section-2: Answering Questions (Input: Context and Question, Output: Answer )
  tokenized_input_qa = tokenizer(context + '</s><s>' +question, truncation=True, max_length=512)['input_ids']
  # task=0 for Question Generation, 1 for Question Answering and 2 for Uncommon Word generation
  task = 1
  tokenized_input_position_qa = torch.tensor(list(range(len(context_tokens)+2)) + list(range(len(question_tokens)+2)))
  task_embedding_input_qa = torch.tensor([task] * len(tokenized_input_qa))
  # Segment ID = 0 for context, 1 for answer and 2 for question. and +2 is for token <s> and </s> for each segment
  segment_embedding_input_qa = torch.tensor([0] * (len(context_tokens)+2) + [2] * (len(question_tokens)+2))

  # Section-3: Uncommon Word Generation (Input: Context, Output: Word Distribution)
  tokenized_input_uw = tokenizer(context, truncation=True, max_length=512)['input_ids']
  # task=0 for Question Generation, 1 for Question Answering and 2 for Uncommon Word generation
  task = 2
  tokenized_input_position_uw = torch.tensor(list(range(len(context_tokens)+2)))
  task_embedding_input_uw = torch.tensor([task] * len(tokenized_input_uw))
  # Segment ID = 0 for context, 1 for answer and 2 for question. and +2 is for token <s> and </s> for each segment
  segment_embedding_input_uw = torch.tensor([0] * (len(context_tokens)+2))

  return {
      'context_tokens': context_tokens,
      'question_tokens': question_tokens,
      'question_input_ids': question_input_ids,
      'quesiton_input_positions': quesiton_input_positions,
      'answer_tokens': answer_tokens,
      'answer_token_start': answer_token_start,
      'answer_token_end': answer_token_end,
      'tokenized_input_qg': tokenized_input_qg,
      'tokenized_input_position_qg': tokenized_input_position_qg,
      'task_embedding_input_qg': task_embedding_input_qg,
      'segment_embedding_input_qg': segment_embedding_input_qg,
      'tokenized_input_qa': tokenized_input_qa,
      'tokenized_input_position_qa': tokenized_input_position_qa,
      'task_embedding_input_qa': task_embedding_input_qa,
      'segment_embedding_input_qa': segment_embedding_input_qa,
      'tokenized_input_uw': tokenized_input_uw,
      'tokenized_input_position_uw': tokenized_input_position_uw,
      'task_embedding_input_uw': task_embedding_input_uw,
      'segment_embedding_input_uw': segment_embedding_input_uw
  }


In [12]:
train_dataset = train_dataset.map(tokenize_single_sample)
test_dataset = test_dataset.map(tokenize_single_sample)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

KeyboardInterrupt: ignored

In [ ]:
print(train_dataset['question_tokens'][0])
print(train_dataset['question_input_ids'][0])

#Model Definition

In [ ]:
class EmbeddingLayer(nn.Module):
  def __init__(self, embedding_dim):
    super(EmbeddingLayer, self).__init__()
    self.embedding_dim = embedding_dim
    self.bart_embedding = AutoModel.from_pretrained(model_name).shared
    # #Converting BART embedding to desired dimension
    # self.word_embedding = nn.Linear(self.bart_embedding.weight.shape[1], embedding_dim)
    #position embedding is done combined with word embedding in bart
    # self.positional_embedding = AutoModel.from_pretrained(model_name).get_input_embeddings().position_embeddings
    self.task_embedding = nn.Embedding(3, embedding_dim)
    self.segment_embedding = nn.Embedding(3, embedding_dim)


  def forward(self, word_input, position_input, task_input, segment_input):
    # print(word_input.shape)
    word_input = self.bart_embedding(word_input)
    # # print(word_input.shape)
    # word_input = self.word_embedding(word_input)
    # print(word_input.shape)
    # position_input = self.positional_embedding(position_input)
    task_input = self.task_embedding(task_input)
    # print(task_input.shape)
    segment_input = self.segment_embedding(segment_input)
    # print(segment_input.shape)

    embedding_output = word_input + (task_input + segment_input) / math.sqrt(self.embedding_dim)
    return embedding_output

In [ ]:
def create_distillation_mask(input_tokens):
  num_rows, num_cols = input_tokens.shape
  percent_true = 0.10
  num_cols -= 2 #subtracting 2 as input id contains <s> and </s>
  # Number of True elements per row
  num_true_per_row = int(num_cols * percent_true)

  # Create and shuffle each row, then stack them into a 2D tensor
  rows = [torch.cat((torch.ones(num_true_per_row, dtype=torch.bool),
                    torch.zeros(num_cols - num_true_per_row, dtype=torch.bool)))[torch.randperm(num_cols)]
          for _ in range(num_rows)]
  b = torch.stack(rows)

  # Add False columns at the beginning and end of each row for <s> and </s>
  b = torch.cat((torch.zeros(num_rows, 1, dtype=torch.bool), b, torch.zeros(num_rows, 1, dtype=torch.bool)), dim=1)

  return b

In [ ]:
class QuesitonGenerationWithKnowledgeDist(nn.Module):
  def __init__(self, embedding_dim, vocab_size, pad_token_id):
    super(QuesitonGenerationWithKnowledgeDist, self).__init__()
    self.embedding_dim = embedding_dim
    self.vocab_size = vocab_size
    self.pad_token_id = pad_token_id

    self.embedding_layer = EmbeddingLayer(embedding_dim = embedding_dim)
    self.primal_dual_encoder = AutoModel.from_pretrained(model_name).encoder
    self.question_decoder = AutoModel.from_pretrained(model_name).decoder
    self.output_layer_qg = AutoModelForSeq2SeqLM.from_pretrained(model_name).lm_head
    self.start_index_ff_qa = nn.Linear(self.embedding_dim, 1)
    self.end_index_ff_qa = nn.Linear(self.embedding_dim*2, 1)
    self.softmax = nn.Softmax(dim = -1)

    self.pretrained_model_no_grad_uw = AutoModel.from_pretrained(model_name)
    self.Wm_uw = nn.Linear(self.embedding_dim, self.vocab_size)



  def forward(self, tokenized_input_qg, tokenized_input_position_qg, task_embedding_input_qg, segment_embedding_input_qg, question_input_ids,
              tokenized_input_qa, tokenized_input_position_qa, task_embedding_input_qa, segment_embedding_input_qa,
              tokenized_input_uw, tokenized_input_position_uw, task_embedding_input_uw, segment_embedding_input_uw):
    #Question Generation Task
    qg_embedded = self.embedding_layer(tokenized_input_qg, tokenized_input_position_qg, task_embedding_input_qg, segment_embedding_input_qg)
    # print(qg_embedded.shape)
    qg_attention_mask = tokenized_input_qg != self.pad_token_id
    # qg_attention_mask = torch.ones_like(tokenized_input_qg)
    # qg_attention_mask[tokenized_input_qg == self.pad_token_id] = 0
    # print(qg_attention_mask.shape)
    qg_encoded_last_hidden_state = self.primal_dual_encoder(inputs_embeds=qg_embedded, attention_mask=qg_attention_mask).last_hidden_state
    # print("Last Layer of encoder =",qg_encoded_last_hidden_state.shape)
    # qg_question_id_attention_mask = torch.ones_like(question_input_ids)
    # # print("Question Input =",question_input_ids.shape)
    # qg_question_id_attention_mask[question_input_ids == self.pad_token_id] =
    qg_question_id_attention_mask = question_input_ids != self.pad_token_id
    qg_decoded_last_hidden_state = self.question_decoder(input_ids=question_input_ids, attention_mask=qg_question_id_attention_mask, encoder_hidden_states=qg_encoded_last_hidden_state).last_hidden_state
    # print("Last Layer of decoder =",qg_decoded_last_hidden_state.shape)
    qg_generated_questions = self.output_layer_qg(qg_decoded_last_hidden_state)
    # print("Feed Forward Layer Output =",qg_generated_questions.shape)



    #Question Answering Task
    qa_embedded = self.embedding_layer(tokenized_input_qa, tokenized_input_position_qa, task_embedding_input_qa, segment_embedding_input_qa)
    qa_attention_mask = tokenized_input_qa != self.pad_token_id
    # qa_attention_mask[tokenized_input_qa == self.pad_token_id] = 0
    qa_encoded_last_hidden_state = self.primal_dual_encoder(inputs_embeds=qa_embedded, attention_mask=qa_attention_mask).last_hidden_state
    # print(qa_encoded_last_hidden_state.shape)
    start_index_logits = self.start_index_ff_qa(qa_encoded_last_hidden_state).squeeze(-1)
    # qa_attention_mask = qa_attention_mask.bool()
    start_index_logits[~qa_attention_mask] = torch.min(start_index_logits)
    start_index_sm = self.softmax(start_index_logits)
    final_start_idx = torch.argmax(start_index_sm, dim=-1, keepdim=True)

    # print(final_start_idx.shape)
    ignore_for_end_index_mask = (torch.arange(qa_attention_mask.size(1)) <= final_start_idx)  | ~qa_attention_mask
    repeated_tensor_of_final_start_index = torch.gather(qa_encoded_last_hidden_state, 1, final_start_idx.unsqueeze(2).expand(-1, qa_encoded_last_hidden_state.size(1), qa_encoded_last_hidden_state.size(2)))
    qa_encoded_last_hidden_state = torch.cat((qa_encoded_last_hidden_state, repeated_tensor_of_final_start_index), dim=-1)
    end_index_logits = self.end_index_ff_qa(qa_encoded_last_hidden_state).squeeze(-1)
    end_index_logits[ignore_for_end_index_mask] = torch.min(start_index_logits)
    end_index_sm = self.softmax(end_index_logits)



    #Uncommon Word Generation
    uw_embedded = self.embedding_layer(tokenized_input_uw, tokenized_input_position_uw, task_embedding_input_uw, segment_embedding_input_uw)
    uw_attention_mask = tokenized_input_uw != self.pad_token_id
    # qa_attention_mask[tokenized_input_qa == self.pad_token_id] = 0
    uw_encoded_last_hidden_state = self.primal_dual_encoder(inputs_embeds=uw_embedded, attention_mask=uw_attention_mask).last_hidden_state
    uw_distillation_mask = create_distillation_mask(tokenized_input_uw)
    # print(uw_distillation_mask)
    with torch.no_grad():
      uw_pretrained_encoded_last_hidden_state = self.pretrained_model_no_grad_uw(input_ids=tokenized_input_uw, attention_mask=uw_distillation_mask, decoder_input_ids = tokenized_input_uw).last_hidden_state
    masked_word_encoding_pre = uw_pretrained_encoded_last_hidden_state.masked_select(uw_distillation_mask.unsqueeze(-1)).view(-1, uw_pretrained_encoded_last_hidden_state.shape[-1])
    y_pre = self.softmax(self.Wm_uw(masked_word_encoding_pre))
    # print(masked_word_encoding_pre.shape)
    masked_word_encoding_en = uw_encoded_last_hidden_state.masked_select(uw_distillation_mask.unsqueeze(-1)).view(-1, uw_encoded_last_hidden_state.shape[-1])
    y_en = self.softmax(self.Wm_uw(masked_word_encoding_en))



    return qg_generated_questions, start_index_logits, end_index_logits, start_index_sm, end_index_sm, y_pre, y_en

#Training_Model

##Custom Collate Function

In [ ]:
def padding_all_tokens(all_tokens):
    max_length = max(len(token_row) for token_row in all_tokens)
    padded_tokens = []
    for token in all_tokens:
        padded_row = token + ['<pad>'] * (max_length - len(token))
        padded_tokens.append(padded_row)
    return padded_tokens

def padding_all_sequences(all_sequences):
    return pad_sequence(all_sequences, batch_first=True, padding_value=tokenizer.pad_token_id)

In [ ]:
def custom_collate(batch):
  context_tokens = padding_all_tokens([item['context_tokens'] for item in batch])

  question_tokens = padding_all_tokens([item['question_tokens'] for item in batch])
  question_input_ids = padding_all_sequences([torch.tensor(item['question_input_ids']) for item in batch])
  quesiton_input_positions = padding_all_sequences([torch.tensor(item['quesiton_input_positions']) for item in batch])

  answer_tokens = padding_all_tokens([item['answer_tokens'] for item in batch])
  answer_token_start = torch.tensor([torch.tensor(item['answer_token_start']) for item in batch])
  answer_token_end = torch.tensor([torch.tensor(item['answer_token_end']) for item in batch])

  tokenized_input_qg = padding_all_sequences([torch.tensor(item['tokenized_input_qg']) for item in batch])
  tokenized_input_position_qg = padding_all_sequences([torch.tensor(item['tokenized_input_position_qg']) for item in batch])
  task_embedding_input_qg = padding_all_sequences([torch.tensor(item['task_embedding_input_qg']) for item in batch])
  segment_embedding_input_qg = padding_all_sequences([torch.tensor(item['segment_embedding_input_qg']) for item in batch])

  tokenized_input_qa = padding_all_sequences([torch.tensor(item['tokenized_input_qa']) for item in batch])
  tokenized_input_position_qa = padding_all_sequences([torch.tensor(item['tokenized_input_position_qa']) for item in batch])
  task_embedding_input_qa = padding_all_sequences([torch.tensor(item['task_embedding_input_qa']) for item in batch])
  segment_embedding_input_qa = padding_all_sequences([torch.tensor(item['segment_embedding_input_qa']) for item in batch])

  tokenized_input_uw = padding_all_sequences([torch.tensor(item['tokenized_input_uw']) for item in batch])
  tokenized_input_position_uw = padding_all_sequences([torch.tensor(item['tokenized_input_position_uw']) for item in batch])
  task_embedding_input_uw = padding_all_sequences([torch.tensor(item['task_embedding_input_uw']) for item in batch])
  segment_embedding_input_uw = padding_all_sequences([torch.tensor(item['segment_embedding_input_uw']) for item in batch])

  return {
      'context_tokens': context_tokens,
      'question_tokens': question_tokens,
      'question_input_ids': question_input_ids,
      'quesiton_input_positions': quesiton_input_positions,
      'answer_tokens': answer_tokens,
      'answer_token_start': answer_token_start,
      'answer_token_end': answer_token_end,
      'tokenized_input_qg': tokenized_input_qg,
      'tokenized_input_position_qg': tokenized_input_position_qg,
      'task_embedding_input_qg': task_embedding_input_qg,
      'segment_embedding_input_qg': segment_embedding_input_qg,
      'tokenized_input_qa': tokenized_input_qa,
      'tokenized_input_position_qa': tokenized_input_position_qa,
      'task_embedding_input_qa': task_embedding_input_qa,
      'segment_embedding_input_qa': segment_embedding_input_qa,
      'tokenized_input_uw': tokenized_input_uw,
      'tokenized_input_position_uw': tokenized_input_position_uw,
      'task_embedding_input_uw': task_embedding_input_uw,
      'segment_embedding_input_uw': segment_embedding_input_uw
  }

#Training_Phase

In [ ]:
model_name = 'facebook/bart-base'

dimension_of_model = AutoModel.from_pretrained(model_name).shared.weight.shape[1]
print(dimension_of_model)

batch_size = 4
num_epochs = 1
lr = 3e-5

alpha = 0.8
beta = 0.15

In [ ]:
store_train_dataset = train_dataset
store_test_dataset = test_dataset

In [ ]:
indices = list(range(20))
train_dataset = store_train_dataset.select(indices)
indices = list(range(10))
test_dataset = store_test_dataset.select(indices)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn= custom_collate)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn= custom_collate)

In [ ]:
model = QuesitonGenerationWithKnowledgeDist(embedding_dim= dimension_of_model, vocab_size = tokenizer.vocab_size, pad_token_id=tokenizer.pad_token_id)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

total_training_steps = len(train_dataloader) * num_epochs
learning_rate_warmup_steps = 50
scheduler = get_linear_schedule_with_warmup(optimizer, learning_rate_warmup_steps, total_training_steps)

best_val_loss = float('inf')
best_model = None

In [ ]:
for epoch in range(num_epochs):
  print(f"Epoch {epoch + 1}/{num_epochs}")
  model.train()
  train_loss = 0
  for batch in tqdm(train_dataloader):
    optimizer.zero_grad()
    gen_qs, start_logits, end_logits, start_smax, end_smax, y_pre, y_en = model(batch['tokenized_input_qg'], batch['tokenized_input_position_qg'], batch['task_embedding_input_qg'], batch['segment_embedding_input_qg'], batch['question_input_ids'],
                                                                                batch['tokenized_input_qa'], batch['tokenized_input_position_qa'], batch['task_embedding_input_qa'], batch['segment_embedding_input_qa'],
                                                                                batch['tokenized_input_uw'], batch['tokenized_input_position_uw'], batch['task_embedding_input_uw'], batch['segment_embedding_input_uw'])
    target_qs = batch['question_input_ids']
    qg_loss = criterion(gen_qs.view(-1, gen_qs.shape[-1]), target_qs.view(-1))

    answer_start_pos = batch['answer_token_start']
    answer_end_pos = batch['answer_token_end']
    print(end_logits)
    print(answer_end_pos)
    qa_loss = criterion(start_logits, answer_start_pos) + criterion(end_logits, answer_end_pos)
    # print(criterion(start_logits, answer_start_pos).item(), criterion(end_logits, answer_end_pos).item())
    uw_loss = -torch.sum(y_en * torch.log(y_pre))
    print(qg_loss.item(), qa_loss.item(), uw_loss.item())
    loss_in_epoch = qg_loss + alpha * qa_loss + beta * uw_loss
    loss_in_epoch.backward()
    optimizer.step()
    scheduler.step()
    print(loss_in_epoch.item())
    train_loss += loss_in_epoch.item()
  print("Train Loss = ",train_loss/len(train_dataloader))

  model.eval()
  val_loss = 0
  with torch.no_grad():
    for batch in tqdm(test_dataloader):
      gen_qs, start_logits, end_logits, start_smax, end_smax, y_pre, y_en = model(batch['tokenized_input_qg'], batch['tokenized_input_position_qg'], batch['task_embedding_input_qg'], batch['segment_embedding_input_qg'], batch['question_input_ids'],
                                                                                batch['tokenized_input_qa'], batch['tokenized_input_position_qa'], batch['task_embedding_input_qa'], batch['segment_embedding_input_qa'],
                                                                                batch['tokenized_input_uw'], batch['tokenized_input_position_uw'], batch['task_embedding_input_uw'], batch['segment_embedding_input_uw'])
      target_qs = batch['question_input_ids']
      qg_loss = criterion(gen_qs.view(-1, gen_qs.shape[-1]), target_qs.view(-1))

      answer_start_pos = batch['answer_token_start']
      answer_end_pos = batch['answer_token_end']
      qa_loss = criterion(start_logits, answer_start_pos) + criterion(end_logits, answer_end_pos)

      uw_loss = -torch.sum(y_en * torch.log(y_pre))

      loss_in_epoch = qg_loss + alpha * qa_loss + beta * uw_loss
      print(loss_in_epoch.item())
      val_loss += loss_in_epoch.item()
  val_loss = val_loss/len(test_dataloader)
  print("Train Loss = ",val_loss)

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    torch.save({
        "model_param": model.state_dict(),
        "optim_param": optimizer.state_dict(),
        "bst_dev_loss": best_val_loss,
        "epoch": epoch,
        "learning_rate": lr},
               f"/content/drive/MyDrive/NLPwithDeepLearning/FinalProject/QwithKD.pth")


In [50]:
temp_dataset = train_dataset[0:1]

In [20]:
dimension_of_model = AutoModel.from_pretrained(model_name).shared.weight.shape[1]
print(dimension_of_model)

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

768


In [51]:
encoder = QuesitonGenerationWithKnowledgeDist(embedding_dim= dimension_of_model, vocab_size = tokenizer.vocab_size, pad_token_id=tokenizer.pad_token_id)

In [52]:
ques, sl, el, ssm, esm, y_pre, y_en = encoder(torch.tensor(temp_dataset['tokenized_input_qg']), torch.tensor(temp_dataset['tokenized_input_position_qg']), torch.tensor(temp_dataset['task_embedding_input_qg']), torch.tensor(temp_dataset['segment_embedding_input_qg']), torch.tensor(temp_dataset['question_input_ids']),
                 torch.tensor(temp_dataset['tokenized_input_qa']), torch.tensor(temp_dataset['tokenized_input_position_qa']), torch.tensor(temp_dataset['task_embedding_input_qa']), torch.tensor(temp_dataset['segment_embedding_input_qa']),
                 torch.tensor(temp_dataset['tokenized_input_uw']), torch.tensor(temp_dataset['tokenized_input_position_uw']), torch.tensor(temp_dataset['task_embedding_input_uw']), torch.tensor(temp_dataset['segment_embedding_input_uw']))

In [53]:
print(ques.shape)

torch.Size([1, 19, 50265])


In [54]:
target_ids = torch.tensor(temp_dataset['question_input_ids'])
print(target_ids.shape)

torch.Size([1, 19])


In [55]:
print(ques.shape[-1])

50265


In [57]:
qg_loss = criterion(ques.view(-1, ques.shape[-1]), target_ids.view(-1))
print(qg_loss)

tensor(7.8833, grad_fn=<NllLossBackward0>)


In [58]:
answer_start = torch.tensor(temp_dataset['answer_token_start'])
answer_end = torch.tensor(temp_dataset['answer_token_end'])

In [60]:
print(answer_start)
print(answer_end)

tensor([117])
tensor([124])


In [61]:
print(sl.shape)
print(el.shape)

torch.Size([1, 181])
torch.Size([1, 181])


In [62]:
qa_loss = criterion(sl, answer_start) + criterion(el, answer_end)
print(qa_loss)

tensor(10.3349, grad_fn=<AddBackward0>)


In [63]:
kd_loss = -torch.sum(y_en * torch.log(y_pre))
print(kd_loss)

tensor(179.3923, grad_fn=<NegBackward0>)


In [64]:
total_loss = qg_loss + 0.8 * qa_loss + 0.15 * kd_loss
print(total_loss)

tensor(43.0601, grad_fn=<AddBackward0>)


In [197]:
primal_dual_embeddings = torch.randint(0, 10, size=(3, 5, 4))
print(primal_dual_embeddings)

tensor([[[5, 7, 6, 1],
         [3, 6, 7, 3],
         [5, 9, 5, 0],
         [6, 3, 0, 5],
         [4, 8, 0, 9]],

        [[4, 2, 1, 9],
         [3, 4, 1, 6],
         [6, 7, 3, 3],
         [8, 3, 5, 0],
         [9, 1, 0, 2]],

        [[0, 7, 3, 1],
         [0, 8, 3, 4],
         [3, 5, 5, 4],
         [1, 7, 5, 2],
         [3, 7, 7, 8]]])


In [199]:
random_values = torch.rand(3, 5)

# Convert random values to boolean (True or False)
input_mask = random_values < 0.30

print(input_mask)

tensor([[False, False, False,  True,  True],
        [ True, False, False, False,  True],
        [ True,  True,  True,  True, False]])


In [213]:
masked_word_embeddings_en = primal_dual_embeddings.masked_select(input_mask.unsqueeze(-1)).view(-1, primal_dual_embeddings.shape[-1])
print(masked_word_embeddings_en)

tensor([[6, 3, 0, 5],
        [4, 8, 0, 9],
        [4, 2, 1, 9],
        [9, 1, 0, 2],
        [0, 7, 3, 1],
        [0, 8, 3, 4],
        [3, 5, 5, 4],
        [1, 7, 5, 2]])


In [202]:
a = primal_dual_embeddings
b = input_mask

In [208]:
print(a.masked_select(b.unsqueeze(-1)))

tensor([6, 3, 0, 5, 4, 8, 0, 9, 4, 2, 1, 9, 9, 1, 0, 2, 0, 7, 3, 1, 0, 8, 3, 4,
        3, 5, 5, 4, 1, 7, 5, 2])


In [209]:
print(a[b.unsqueeze(-1)])

IndexError: ignored

In [206]:
print(a.masked_select(b.unsqueeze(-1)).reshape(-1,a.shape[1]))

RuntimeError: ignored

In [212]:
print(a.shape[-1])

4
